In [1]:
import anndata
import scanpy as sc
import numpy as np

import warnings
# warnings.filterwarnings("ignore")

In [2]:
# 指定文件夹路径
folder_path = "/home/dongjiayi/workbench/denoise/data/DEG/v4/seed1/"

In [3]:
raw_adata = sc.read(folder_path+"demo.h5ad")

/mnt/sdc/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
raw_adata

AnnData object with n_obs × n_vars = 1327 × 2000
    obs: 'tech', 'cellstate', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'hvg', 'log1p', 'rank_genes_groups'

In [5]:
sequence = list(range(raw_adata.shape[0]))  # 序列从1到1327

# 定义要随机选择的数据数量
num_samples = 100  # 示例：随机选择10个数据

# 从序列中随机选择数据
random_samples = np.random.choice(sequence, num_samples, replace=False)

print(random_samples)

[ 106 1059  672  731 1056 1257  383  856  291 1008 1230  289 1045  694
  216  646  141  112  906  258 1175  474  160  742 1220  301  354  147
  262  549   40  409  538  360  587 1214  819  918  692  349  756  101
  546  233 1113  278  246  678  707  732 1116  930  134 1067  793 1205
  947  681  699  341 1144  204 1028  423  175 1049 1087  362  963  759
 1108 1118  658 1182  284  754   35 1298 1014  224 1052   27  363  784
  857   69  495 1325  818 1104  844  683  156  633  636  126  364  517
  935 1278]


In [6]:
def get_deg_list(adata, cell_type: str = "1", threshold: float = 0.05):
    sc.tl.rank_genes_groups(adata, cell_type, method="wilcoxon")
    deg = {}
    for i in set(adata.obs[cell_type]):
        gene_list = adata.uns["rank_genes_groups"]["names"][str(i)]
        msk = adata.uns["rank_genes_groups"]["pvals"][str(i)] <= threshold
        deg[str(i)] = list(gene_list[msk])
    return deg

def get_topn_deg_list(adata, cell_type: str = "1", n_genes : int=2000):
    sc.tl.rank_genes_groups(adata, cell_type, n_genes=n_genes, method="wilcoxon")
    deg = {}
    for i in set(adata.obs[cell_type]):
        gene_list = adata.uns["rank_genes_groups"]["names"][str(i)]
        deg[str(i)] = list(gene_list)
    print(len(gene_list))
    return deg

adata = sc.read_csv("/home/dongjiayi/workbench/denoise/data/DEG/v4/GSE108382_Melanoma_bulk.txt", delimiter='\t').T
print(adata.obs_names)
adata.obs['cellstate'] = ['par', 'par', 'br', 'br', 'par', 'par', 'br', 'br']
adata = adata[:, raw_adata.var_names]
print(adata)

# sc.pp.normalize_total(adata, target_sum=10000)
# sc.pp.log1p(adata)

gene_list = get_deg_list(adata, 'cellstate')

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: overflow encountered in expm1
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:420: RuntimeWarning: overflow encountered in expm1
  self.expm1_func(mean_rest) + 1e-9
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:419: RuntimeWarning: invalid value encountered in true_divide
  foldchanges = (self.expm1_func(mean_group) + 1e-9) / (
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: divide by zero encountered in log2

Index(['451Lu_bulk_parental_rep1', '451Lu_bulk_parental_rep2',
       '451Lu_bulk_resistant_rep1', '451Lu_bulk_resistant_rep2',
       'A375_bulk_parental_rep1', 'A375_bulk_parental_rep2',
       'A375_bulk_resistant_rep1', 'A375_bulk_resistant_rep2'],
      dtype='object')
View of AnnData object with n_obs × n_vars = 8 × 2000
    obs: 'cellstate'


In [7]:
def calculate_marker_matches(adata, topn, gene_list, raw_adata):
    adata.obs['cellstate'] = raw_adata.obs['cellstate'] 
    adata.obs['batch'] = raw_adata.obs['batch'] 
    # print(adata.shape)
    # for i in gene_list.keys():
    #     gene_list[i] = gene_list[i][:topn]
    sc.tl.rank_genes_groups(adata, 'cellstate')
    marker_matches_all = sc.tl.marker_gene_overlap(adata, gene_list, top_n_markers=topn)
    # print(marker_matches_all)

    temp = adata[adata.obs['batch'] == 0]
    # print(temp.shape)
    sc.tl.rank_genes_groups(temp, 'cellstate')
    marker_matches_1 = sc.tl.marker_gene_overlap(temp, gene_list, top_n_markers=topn)
    # print(marker_matches_1)
    
    temp = adata[adata.obs['batch'] == 1]
    # print(temp.shape)
    sc.tl.rank_genes_groups(temp, 'cellstate')
    marker_matches_2 = sc.tl.marker_gene_overlap(temp, gene_list, top_n_markers=topn)
    # print(marker_matches_2)
    return marker_matches_all, marker_matches_1, marker_matches_2

In [8]:
import pandas as pd
folder = "/home/dongjiayi/workbench/denoise/data/DEG/v4/"
# 创建一个空的DataFrame或含有列名的DataFrame
df = pd.DataFrame(columns=['methods', 'cell type', 'batch_info', 'top-n', 'score'])
count = 0

for i in range(10):
    for j in [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]:
            folder_path = folder + f'seed{i}/'
            
            raw_adata = sc.read(folder_path+"demo.h5ad")[random_samples]
            
            # raw data
            adata = raw_adata
            marker_matches_all, marker_matches_0, marker_matches_1 = calculate_marker_matches(adata, j, gene_list, raw_adata)
            df.loc[count] = ['raw', 'br', 'all', str(j), marker_matches_all['br']['br']]
            count+=1
            df.loc[count] = ['raw', 'par', 'all', str(j), marker_matches_all['par']['par']]
            count+=1
            df.loc[count] = ['raw', 'br', '0', str(j), marker_matches_0['br']['br']]
            count+=1
            df.loc[count] = ['raw', 'par', '0', str(j), marker_matches_0['par']['par']]
            count+=1
            df.loc[count] = ['raw', 'br', '1', str(j), marker_matches_1['br']['br']]
            count+=1
            df.loc[count] = ['raw', 'par', '1', str(j), marker_matches_1['par']['par']]
            count+=1
            
            for method in ['DGDM', 'MAGIC', 'DeepImpute', 'SCALEX', 'scDisInFact_0']:
                # DGDM
                adata = sc.read(folder_path+f"denoise_adata_{method}.h5ad")[random_samples]
                # adata.X[adata.X < 0] = 0
                # if method == "DeepImpute" or method == "MAGIC":
                #     sc.pp.normalize_total(adata, target_sum=10000)
                #     sc.pp.log1p(adata)
                marker_matches_all, marker_matches_0, marker_matches_1 = calculate_marker_matches(adata, j, gene_list, raw_adata)
                df.loc[count] = [method, 'br', 'all', str(j), marker_matches_all['br']['br']]
                count+=1
                df.loc[count] = [method, 'par', 'all', str(j), marker_matches_all['par']['par']]
                count+=1
                df.loc[count] = [method, 'br', '0', str(j), marker_matches_0['br']['br']]
                count+=1
                df.loc[count] = [method, 'par', '0', str(j), marker_matches_0['par']['par']]
                count+=1
                df.loc[count] = [method, 'br', '1', str(j), marker_matches_1['br']['br']]
                count+=1
                df.loc[count] = [method, 'par', '1', str(j), marker_matches_1['par']['par']]
                count+=1

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_743189/1891745296.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cellstate'] = raw_adata.obs['cellstate']


/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_743189/1891745296.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cellstate'] = raw_adata.obs['cellstate']
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_gene

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_743189/1891745296.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cellstate'] = raw_adata.obs['cellstate']
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.p

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_743189/1891745296.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cellstate'] = raw_adata.obs['cellstate']
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_gene

/tmp/ipykernel_743189/1891745296.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cellstate'] = raw_adata.obs['cellstate']
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_743189/1891745296.py:2: ImplicitModificationWarning: Trying to modify attribu

...

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_743189/1891745296.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cellstate'] = raw_adata.obs['cellstate']
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.p

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_743189/1891745296.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cellstate'] = raw_adata.obs['cellstate']
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.p

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_743189/1891745296.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cellstate'] = raw_adata.obs['cellstate']
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_743189/1891745296.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cellstate'] = raw_adata.obs['cellstate']
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_gene

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/tmp/ipykernel_743189/1891745296.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cellstate'] = raw_adata.obs['cellstate']
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_gene

In [9]:
df.to_csv('temp.csv')

In [10]:
df.groupby(['methods', 'cell type', 'batch_info', 'top-n']).mean().to_csv('temp.csv')

# condition

In [11]:
import copy
# 指定文件夹路径
folder = "/home/dongjiayi/workbench/denoise/data/DEG/v4/"

In [12]:
def de_num_cal(adata, j, raw_adata):
    de_num_list = {}
    adata.obs_names = raw_adata.obs_names
    adata.obs['cellstate'] = raw_adata.obs['cellstate'] 
    adata.obs['batch'] = raw_adata.obs['batch'] 
    for i in set(adata.obs['cellstate']):
        # print(i)
        tdata = adata[adata.obs['cellstate'] == i]
        sc.tl.rank_genes_groups(tdata, 'batch')

        temp = tdata.uns["rank_genes_groups"]["names"]['0']
        msk = tdata.uns["rank_genes_groups"]["pvals"]['0'] <= j
        # print(len(list(temp[msk])))
        de_num_list[i] = len(list(temp[msk]))
    return de_num_list

In [13]:
import pandas as pd
folder = "/home/dongjiayi/workbench/denoise/data/DEG/v4/"
# 创建一个空的DataFrame或含有列名的DataFrame
df = pd.DataFrame(columns=['methods', 'cell type', 'p_values', 'score'])
count = 0

for i in range(10):
    for j in [0, 0.002, 0.004, 0.006, 0.008, 0.02, 0.04, 0.06, 0.08, 0.1]:
            folder_path = folder + f'seed{i}/'
            
            raw_adata = sc.read(folder_path+"demo.h5ad")
            
            # raw data
            adata = raw_adata
            de_num_list = de_num_cal(adata, j, raw_adata)
            df.loc[count] = ['raw', 'br',  str(j), de_num_list['br']]
            count+=1
            df.loc[count] = ['raw', 'par', str(j), de_num_list['par']]
            count+=1

            
            for method in ['DGDM', 'MAGIC', 'DeepImpute', 'SCALEX', 'scDisInFact_0']:
                # DGDM
                adata = sc.read(folder_path+f"denoise_adata_{method}.h5ad")
                # adata.X[adata.X < 0] = 0
                # if method == "DeepImpute" or method == "DeepImpute":
                #     sc.pp.normalize_total(adata, target_sum=10000)
                #     sc.pp.log1p(adata)        
                de_num_list = de_num_cal(adata, j, raw_adata)
                df.loc[count] = [method, 'br',  str(j), de_num_list['br']]
                count+=1
                df.loc[count] = [method, 'par', str(j), de_num_list['par']]
                count+=1

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/pytho

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/pytho

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/pytho

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_group

...

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/pytho

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/py

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scan

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/dongjiayi/miniconda3/envs/python38/lib/pytho

In [14]:
df.groupby(['methods', 'cell type', 'p_values']).mean().to_csv('temp.csv')

In [15]:
folder_path = folder + 'seed0/'
raw_adata = sc.read(folder_path+"demo.h5ad")
# raw data
adata = copy.deepcopy(raw_adata)
adata.obs['cellstate'] = raw_adata.obs['cellstate'] 
adata.obs['batch'] = raw_adata.obs['batch'] 
# sc.pp.scale(adata)
# sc.pp.combat(adata, key='batch')
de_num_cal(adata)

/home/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/anndata/_core/anndata.py:1838: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


TypeError: de_num_cal() missing 2 required positional arguments: 'j' and 'raw_adata'

In [ ]:
folder_path = folder + 'seed0/'
adata = sc.read(folder_path+"denoise_adata_DGDM.h5ad")

# adata.X[adata.X < 0] = 0

adata.obs['cellstate'] = raw_adata.obs['cellstate'] 
adata.obs['batch'] = raw_adata.obs['batch'].apply(lambda x:str(x)) 
# sc.pp.scale(adata)
# sc.pp.combat(adata, key='batch')
sc.tl.rank_genes_groups(adata, 'batch')

de_num_cal(adata)

{'par': [0, 682, 769, 816, 857, 897], 'br': [0, 649, 710, 767, 818, 854]}

In [ ]:
folder_path = folder + 'seed0/'
adata = sc.read(folder_path+"denoise_adata_SCALEX.h5ad")
adata.obs['cellstate'] = raw_adata.obs['cellstate'] 
adata.obs['batch'] = raw_adata.obs['batch'].apply(lambda x:str(x)) 
sc.tl.rank_genes_groups(adata, 'batch')

de_num_cal(adata)

{'par': [3, 923, 990, 1045, 1096, 1133], 'br': [2, 849, 925, 976, 1013, 1044]}

In [ ]:
folder_path = folder + 'seed0/'
adata = sc.read(folder_path+"denoise_adata_MAGIC.h5ad")

adata.obs['cellstate'] = raw_adata.obs['cellstate'] 
adata.obs['batch'] = raw_adata.obs['batch'].apply(lambda x:str(x)) 
sc.tl.rank_genes_groups(adata, 'batch')

de_num_cal(adata)

{'par': [6, 1843, 1861, 1891, 1900, 1907],
 'br': [2, 1806, 1830, 1865, 1880, 1886]}

In [ ]:
folder_path = folder + 'seed0/'
adata = sc.read(folder_path+"denoise_adata_DeepImpute.h5ad")

adata.obs['cellstate'] = raw_adata.obs['cellstate'] 
adata.obs['batch'] = raw_adata.obs['batch'].apply(lambda x:str(x)) 
sc.tl.rank_genes_groups(adata, 'batch')

de_num_cal(adata)

{'par': [0, 909, 971, 1015, 1058, 1091], 'br': [0, 874, 950, 1000, 1042, 1078]}

In [ ]:
folder_path = folder + 'seed0/'
adata = sc.read(folder_path+"denoise_adata_scDisInFact_0.h5ad")

adata.obs['cellstate'] = raw_adata.obs['cellstate'] 
adata.obs['batch'] = raw_adata.obs['batch'].apply(lambda x:str(x)) 
sc.tl.rank_genes_groups(adata, 'batch')

de_num_cal(adata)

{'par': [0, 1894, 1905, 1915, 1920, 1921],
 'br': [0, 1725, 1760, 1787, 1801, 1812]}